In [1]:
data_dir = "/home/matt/Documents/brainlit/benchmarking_datasets/"
swc_dir = data_dir + "Manual-GT/"

In [2]:
## Match image and SWC files
im_name = "validation_1-gfp.tif" # choose the file name
swc_name = "10-01_validation_1-5/10-01_validation_1/tree_1.swc" # choose the tree within the file (there may be many SWCs for one image)
vol_offset = [3944.427317, 1689.489974, 2904.058044] # Use the volume offset [x1, y1, z1] from https://docs.google.com/spreadsheets/d/1DtYB-O0CjoPQcxQq_N6kVgeHV91Hmz9ac1HlhfX95og/edit#gid=1836493523

In [3]:
## HYPERPARAMETERS
tube_radius = 2 # value > 0. The radius of the tube for the tube segmentation. Higher radius results in a larger colored region.
region_growing_multiplier = 0.1 # value > 0, usually not more than 1. The multiplier for region growing. Higher multipler results in a larger colored region.
skip = 20 # use every spip'th point to generate the segmentation; e.g. every 2nd point or every 20th point

In [4]:
import brainlit
from brainlit.utils.ngl_pipeline import NeuroglancerSession
from brainlit.viz.swc import *
from brainlit.viz.visualize import *
from brainlit.algorithms.generate_fragments import tube_seg, adaptive_thresh
import napari
from skimage import io
import pandas as pd
import linecache

Using TensorFlow backend.
/home/matt/anaconda3/lib/python3.7/site-packages/napari/__init__.py:38: UserWarning: 
    napari was tested with QT library `>=5.12.3`.
    The version installed is 5.12.1. Please report any issues with this
    specific QT version at https://github.com/Napari/napari/issues.
    
  warn(message=warn_message)


In [5]:
def napari_viewer(img, labels=None, shapes=None, label_name="Segmentation"):
    #with napari.gui_qt():
    viewer = napari.view_image(np.squeeze(np.array(img)))
    if labels is not None:
        viewer.add_labels(labels, name=label_name)
    if shapes is not None:
        viewer.add_shapes(data=shapes, shape_type='path', edge_color='blue', name='Skeleton')
    return viewer

In [6]:
def transform(point, scales, swc_offset, brain_offset, vol_offset):
    im_offset = np.add(brain_offset, vol_offset)
    offset_diff = np.subtract(swc_offset, im_offset)
    return (point + offset_diff)/scales*1000

In [7]:
def create_seed(subneuron_df, scales, swc_offset, brain_offset, vol_offset):
    seed = []
    for index, row in subneuron_df.iterrows():
        point = np.array((row['x'], row['y'], row['z']))
        point = transform(point, scales, swc_offset, brain_offset, vol_offset)
        _, s = adaptive_thresh.get_seed(point)
        seed.append(s)
    #print(seed)
    return seed

In [8]:
im_path = data_dir + im_name
swc_path = swc_dir + swc_name

In [9]:
scales = np.array([298.66187, 301.37174, 1050.67223])

In [10]:
# The SWC offset is at the top of the SWC file
swc_offset = [float(i) for i in linecache.getline(swc_path, 2).split()[2:5]]

if im_name[0] == "v":
    brain_offset = [69445.19581378, 12917.40798423, 30199.63896704] # validation offset
elif im_name[0] == "t":
    brain_offset = [70093.27584462, 15071.5958194, 29306.73645404] # test offset

In [11]:
im = io.imread(im_path).T

In [13]:
%gui qt
napari_viewer(im)

In [14]:
subneuron_df = pd.read_csv(swc_path, skiprows = 3, header = None, sep = " ")
subneuron_df.columns = ["sample",  "structure",      "x",      "y",      "z",
                         "r",  "parent"]

In [15]:
G = df_to_graph(subneuron_df)
print('Number of nodes:', len(G.nodes))
print('Number of edges:', len(G.edges))
print('\n')
print('Sample 1 coordinates (x,y,z)')
G.nodes[1]

Number of nodes: 127
Number of edges: 126


Sample 1 coordinates (x,y,z)


{'x': 13, 'y': -30, 'z': -13}

In [16]:
paths = graph_to_paths(G)
len(paths)

1

In [17]:
transformed_paths = []
for i in range(len(paths)):
    transformed_paths.append(transform(paths[i], scales, swc_offset, brain_offset, vol_offset))

In [18]:
seed = create_seed(subneuron_df, scales, swc_offset, brain_offset, vol_offset)

In [23]:
seed = [s for s in seed if s[0] >= 0 and s[1] >= 0 and s[2] >= 0] ## NOTE: Can't have negative coordinates in confidence connected threshold

In [21]:
%gui qt
napari_viewer(im, labels = labels, shapes=transformed_paths)

In [20]:
labels = tube_seg.tubes_seg(im, transformed_paths[0][::skip], 2, spheres=False)

In [22]:
%gui qt
viewer = napari_viewer(im, labels=labels, shapes=transformed_paths, label_name="Tube Segmentation")

In [24]:
labels = adaptive_thresh.confidence_connected_threshold(im, seed[::skip], num_iter=1, multiplier=region_growing_multiplier)

In [25]:
%gui qt
viewer = napari_viewer(im, labels=labels, shapes=transformed_paths, label_name="Region Growing")